# Metadata Merge - Summary

## Objective
Merge sample metadata (assay versions, demographics) with PLINK .psam files for downstream analysis.

## Key Operations
- Joined `tapestry_metadata.csv` with `tapestry_cds.psam` on sample IDs
- Added assay version labels (v2, v3, v4, v6) to each sample
- Verified no missing joins (all 97,422 samples matched)

## Critical Metadata
- **Assay version**: Later found to be source of batch effects in PC2
- **Sample IDs**: Consistent formatting for cross-file matching
- **Demographics**: Basic covariates for downstream fairness analysis

## Output
- `tapestry_cds_withmeta.psam` - Updated PLINK sample file with metadata columns

## Next Step
Validate LD structure in CDS variant data (Notebook 01).

In [ ]:
import os
import subprocess

import pandas as pd

from google.cloud import bigquery
import google.auth.transport.requests
import pandas_gbq

# Connect to CGP.

- If you need to login, run `gcloud auth application-default login --no-launch-browser`

In [ ]:
creds, project = google.auth.default()
bq_client = bigquery.Client(project=project, credentials=creds)

In [ ]:
job = bq_client.query("SELECT * FROM `aif-usr-p-cbr-genai-1524.EDA.tapestry_inference_only`")
df = job.to_dataframe()
df.shape

In [ ]:
cols = [
    'tap_kitid',
    
    'inferred_sex', 
    'sex_at_birth', 

    'ethnicity',
    'is_hispanic', 
    'race', 
    'gnomad_race', 
    
    'ehr_birthdate', 
    'age_at_sequence',  
        
    'tap_state_enrolled', 
    'tap_state_current', 
    'currently_consented', 
    
    'vcf_format_version',
    'vcf_genome_version',
    'vcf_assay_version',
    
    'has_ra',
    'has_ibd', 
    'has_ibd_crohns', 
    'has_ibd_uc', 
    'has_fh', 
    'has_prediabetes',
    'has_diabetes1', 
    'has_diabetes2', 
    'has_breast_cancer',
    'has_prostate_cancer', 
    'has_high_risk_prostate_cancer',
    'prostate_risk_score',
    
    'max_ldl',
    'height_cm',
    'weight_kg', 
    'bmi', 
    
    'education_level',
    'employment_status',
    'alcohol_oz_per_wk', 
    'has_alcohol_oz_per_wk', 
    'alcohol_oz_per_wk_mean',
    'smoking_years', 
    'smoking_packyears', 
    'smoking_status',

    'ALCOHOL_ABUSE', 
    'ALCOHOL_USE', 
    'ANEMIA', 
    'ANXIETY', 
    'ASTHMA',
    'BIPOLAR', 
    'CANCER', 
    'CARDIAC_ARRHYTHMIA', 
    'CEREBROVASCULAR_DISEASE',
    'CHRONIC_BACK_PAIN', 
    'CONGESTIVE_HEART_FAILURE', 
    'COPD',
    'CORONARY_ARTERY_DISEASE', 
    'DEMENTIA', 
    'DEPRESSION', 
    'DIABETES',
    'DIVERTICULAR_DISEASE', 
    'DRUG_ABUSE', 
    'FIBROMYALGIA', 
    'GERD', 
    'GOUT',
    'GYNECOLOGIC_DISORDER', 
    'HEADACHES', 
    'HEARING_LOSS', 
    'HYPERLIPIDEMIA',
    'HYPERTENSION',
	'HYPERTHYROIDISM',
	'INFLAMMATORY_SKIN_DISEASE',
    'INTERSTITIAL_LUNG_DISEASE',
	'IRRITABLE_BOWEL_DISEASE',
	'LIVER_DISEASE',
    'NEUROPATHY',
	'OBESITY',
	'OSTEOARTHRITIS',
	'OSTEOPOROSIS',
    'PARKINSONS_DISEASE',
	'PULMONARY_CIRCULATION_DISORDER',
    'PEPTIC_ULCER_DISEASE',
	'PERIPHERAL_VASCULAR_DISEASE',
    'PROSTATE_HYPERPLASIA',
	'PTSD',
	'RENAL_DISEASE',
    'SEVERE_VISION_REDUCTION',
	'SLEEP_DISORDER',
    'URINARY_INCONTINENCE',
    'VALVULAR_HEART_DISEASE',

]

In [ ]:
mask = df['tap_kitid'] != ' '
df_tapestry = df.loc[mask, cols].copy()
df_tapestry.shape

In [ ]:
df_tapestry.head()
cols = {'tap_kitid': '#IID'}
df_tapestry.rename(columns=cols, inplace=True)

df_tapestry.columns = df_tapestry.columns.str.upper()
df_tapestry = df_tapestry.sort_values('#IID').reset_index(drop=True)
df_tapestry.head()

In [ ]:
path = '/home/ext_meehl_joshua_mayo_edu/pre-phd-genomics/02_genomics_domain/data/plink/tapestry/genome_wide/genome_qc.psam'
df_psam = pd.read_csv(path, delim_whitespace=True)
cols = {'SEX': 'INFERRED_SEX_2'}
df_psam.rename(columns=cols, inplace=True)
df_psam.head()

In [ ]:
df_merge = pd.merge(df_psam, df_tapestry, on='#IID', how='inner').copy()
df_merge.insert(1, 'SEX', df_merge['SEX_AT_BIRTH'])
df_merge.head()

In [ ]:
path = '/home/ext_meehl_joshua_mayo_edu/pre-phd-genomics/02_genomics_domain/data/plink/tapestry/genome_wide/metadata.csv'
df_merge.to_csv(path, sep=',', index=False)

path = '/home/ext_meehl_joshua_mayo_edu/pre-phd-genomics/02_genomics_domain/data/plink/tapestry/genome_wide/metadata.psam'
df_merge.to_csv(path, sep='\t', index=False)